In [1]:
from pprint import pprint

import itertools as itls

In [2]:
import pickle

In [20]:
import pandas as pd

import cytoolz as tlz
from cytoolz import curried as tlzc
head = lambda l: list(tlz.take(4, l))

from gensim import corpora

In [21]:
from text2math import raw2text as r2t
from text2math import text2tokens as t2t
from student_topic import mkcorpus as mc

---

In [22]:
DICTIONARY = corpora.Dictionary.load("./student_titles_dict.dict")

---

In [5]:
STUDENT_TITLES = pd.DataFrame.from_csv("../student_title.csv")

In [6]:
STUDENT_MERGED = pd.DataFrame.from_csv("../student_merged.csv")

In [7]:
len(STUDENT_TITLES)

65023

In [8]:
len(STUDENT_MERGED)

22428

---

# Merge On Email Hash

In [9]:
MERGED = STUDENT_MERGED.merge(STUDENT_TITLES, how='inner', left_index=True, right_index=True)

# Make DataFrame For Eeach Faculty

In [10]:
FACULTY_DFS = {faculty: MERGED[MERGED.faculty == faculty] for faculty in MERGED.faculty.unique()}

---

In [11]:
ts = [title
        for faculty, df in FACULTY_DFS.items()
         for title in df.title]

In [12]:
h = mc.uni_and_bigram_cleanup(ts)

In [13]:
FTOKENS = {faculty: mc.uni_and_bigram_cleanup(df.title) for faculty, df in FACULTY_DFS.items()}

In [14]:
FTOKENS[FTOKENS.keys()[0]][0]

(u'antecedents',
 u'implications',
 u'disruptive',
 u'innovation',
 u'evidence',
 u'china',
 u'antecedents_implications',
 u'implications_disruptive',
 u'disruptive_innovation',
 u'innovation_evidence',
 u'evidence_china')

In [27]:
FCORPUS = {faculty: list(tlz.map(DICTIONARY.doc2bow, tokenssets)) for faculty, tokenssets in FTOKENS.items()}

In [36]:
FCORPUS[FCORPUS.keys()[0]][0]

[(129, 1),
 (3972, 1),
 (4980, 1),
 (7023, 1),
 (8502, 1),
 (8525, 1),
 (9128, 1),
 (11563, 1),
 (13039, 1),
 (19432, 1),
 (19722, 1)]

In [37]:
with open("./corpus_by_faculty.pickle", "w+") as f:
    pickle.dump(FCORPUS, f)